In [ ]:
from dscim.menu.simple_storage import Climate, EconVars
import yaml
import xarray as xr
import dscim.menu.baseline
import dscim.menu.risk_aversion
import dscim.menu.equity
from pathlib import Path

import os

USER = os.getenv("USER")
import dask

dask.config.set(**{"array.slicing.split_large_chunks": False})
from itertools import product

MENU_OPTIONS = {
    "adding_up": dscim.menu.baseline.Baseline,
    "risk_aversion": dscim.menu.risk_aversion.RiskAversionRecipe,
    "equity": dscim.menu.equity.EquityRecipe,
}

In [11]:
config_file_path = "configs/dummy_config.yaml"

with open(config_file_path) as stream:
        conf = yaml.safe_load(stream)

In [ ]:
sector = "dummy_not_coastl_sector"
pulse_year = 2020
menu_disc = ['adding_up', 'constant']
eta_rho = [2.0,0.0001]
USA = False
AR = 6
mask = None
order = "scc"
fair_dims = ["simulation"]


menu_option, discount_type = menu_disc
save_path = f"{conf['paths'][f'AR{AR}_ssp_results']}/{sector}/{pulse_year}/"

if mask is not None:
    save_path = save_path + "/" + mask
else:
    save_path = save_path + "/" + "unmasked"

if fair_dims != ["simulation"]:
    save_path = (
        save_path
        + "/"
        + f"fair_collapsed_{'_'.join([i for i in fair_dims if i!='simulation'])}"
    )

if USA:
    econ = EconVars(path_econ=conf["econdata"]["USA_ssp"])
else:
    econ = EconVars(path_econ=conf["econdata"]["global_ssp"])

add_kwargs = {
    "econ_vars": econ,
    "climate_vars": Climate(
        **conf[f"AR{AR}_ssp_climate"],
        pulse_year=pulse_year,
        ecs_mask_name=mask,
    ),
    "formula": conf["sectors"][sector if not USA else sector[:-4]]["formula"],
    "discounting_type": discount_type,
    "sector": sector,
    "ce_path": f"{conf['paths']['reduced_damages_library']}/{sector}/",
    "save_path": save_path,
    "eta": eta_rho[0],
    "rho": eta_rho[1],
    "fair_dims": fair_dims,
}

kwargs = conf["global_parameters"].copy()
for k, v in add_kwargs.items():
    assert (
        k not in kwargs.keys()
    ), f"{k} already set in config. Please check `global_parameters`."
    kwargs.update({k: v})

if "CAMEL" in sector:
    kwargs.update(
        {
            "damage_function_path": f"{conf['paths']['ssp_damage_function_library']}/{sector}/2020/unmasked",
            "save_files": [
                "damage_function_points",
                "marginal_damages",
                "discount_factors",
                "uncollapsed_sccs",
                "scc",
            ],
        }
    )

menu_item = MENU_OPTIONS[menu_option](**kwargs)
menu_item.order_plate(order)

In [ ]:
menu_item.uncollapsed_sccs